# Modern GL

Este notebook contendra el tutorial de la librería de python que implementa __Modern GL__ y notas de la
[documentacion oficial](https://moderngl.readthedocs.io), además de ejemplos prácticos y de la clase.

Se autodefine como 
> Un modulo de rendering de alto rendimiento para Python

e implementa la versión moderna de __OpenGL__ en comparación con __PyOpenGL__ que usa su implementación anterior.

### Instalacion

Se puede instalar desde `pip` con la sentencia
```console
$ pip install ModernGL
```

## The Guide

#### Para empezar

Para renderizar algo se necesita usar desde un `VertexArray`.
 
 1. Estos pueden ser creados desde un objeto `Program` y desde un `Buffer`. 
 2. Para poder crear un `Program` es necesario suministrar los __shaders__ como _strings_. 
 3. Despues se debe usar  el `Buffer` con la informacion pertinente, que sera trasladada a un `VertexArray`. 
 4. Y finalmente renderizar todo esto con el metodo `VertexArray.render()`. 

Todos los objetos e instancias anteriores deben ser creados desde un objeto `Context`.

### Creacion de contextos

Un objeto `context` le da a __ModernGL__ acceso a las instrucciones de __OpenGL__. A pesar de que `context` se debe crear de manera distinta dependiendo del sistema operativo, __ModernGL__ se encarga de seleccionar la opcion por _default_ del sistema operativo.

Luego, los _backend_ de los principales sistemas operativos (con los que trabaja __ModernGL__) permiten dos modos
 - Dectectar una libreria existente para manejar ventanas
```python
import moderngl
# Crear context
ctx = moderngl.create_context()
print('El framebuffer por defecto es: ', ctx.screen)
```
 - Crear un `context` sin encabezado (Sin ventana visible)
```python
import moderngl
# Crear context
ctx = moderngl.create_context(standalone=True)
# Crear framebuffer para renderizar
fbo = ctx.simple_framebuffer((100, 100), 4)
fbo.use()
```

#### Version OpenGL

__ModernGL__ solo soporta 3.3+ `contexts`. En caso de que necesitemos una version especifica de __OpenGL__, podemos pasarle el argumento `require= version`, donde `version` es el argumento de la version minima solicitada.


#### Backend especifico

Para definir el _backend_ que usara nuestro programa (modificar el que esta por defecto), se le puede pasar el argumento `backend= 'bck'`, donde `bck` es el motor de ventanas que se desea utilizar.



In [4]:
import moderngl

import glfw # Backebd para crear contexto
if glfw.init(): # Iniciamos GLFW
    ventana = glfw.create_window(700, 500, 
            "Tarea 1 de Graficos por Computadora en Python", 
            None, None) # Creamos ventana
    # Establecer contexto con la ventana
    glfw.make_context_current(ventana)
    
# Crear context
ctx = moderngl.create_context()
print('El framebuffer por defecto es: ', ctx.screen)

glfw.terminate()

El framebuffer por defecto es:  <Framebuffer: 0>


### Buffer Format

> __VBO__ Vertex Buffer Object
>
> __VAO__ Vertex Array Objects

El __VBO__ es el formato en que guardamos informacion en un `Buffer`, esto se guarda en un _strig_ con formato especificado. 

El __VBO__ esta compuestos por estructuras _C-likes_. Ademas describe la apariencia y el formato de cada uno de los elementos. Un ejemplo de esto podria ser cuando queremos guardar cordenadas _2D_, este __VBO__ tendria la forma `"2f8"`, donde cada elemneto del arreglo consiste de dos _floats_ de 8 bits.

El _formato del Buffer_ es usado en el constructor de `Context.vertex_array()`.

#### Sintaxis

El formato del buffer debe ser como sigue
```
 [count]type[size] [[count]type[size] ... ] [/usage]
```
Donde:
 - `count` es un entero opcional, si se omite se considera 1
 - `type` es un caracter que indica el tipo de dato
     + `f` float
     + `i` int
     + `u` unsigned int
     + `x` padding
 - `size` Numero opcional que representa la cantidad de bytes que se usaran para guardar el tipo.
 
 Este patron se puede repetir varias veces para indicar distintos tipos de datos a guardar. Esto se debe seguir de
 - `/usage` Que consiste en un caracter que indica la manera en que los valores sucesivos deven ser tratados por el shader.
     + `/v` por vertice. Cada uno de los siguientes valores del `buffer` seran suministrados a cada vertice.
     + `/i` por instancia. Los siguientes valores del `buffer` son pasados a cada instancia.
     + `/r` por render. El primer valor del `buffer` sera pasado a cada vertice e instancia.
     
Es importante notar que la combinacion `f1` tiene propiedades curiosas:
 
 1. Corresponde a un flotante, pero corresponde a un `unsigned bytes`. Estos valores son normalizados, lo que implica que los valores de 0 a 255 son transformados a valores flotantes de 0.0 a 1.0 cuando llegan al `vertex shader`. Se espera que sean usados para pasar colores.
 2. Tres de estas instancias `3f1` deben ser asignadas a un atributo `vec3`. Pero en ModernGL 6.0 tambien pueden pasarse a un atributo `vec4`, esto tine la intencion de pasar los colores en formato RGB con un canal alpha.

Este formato es especifico a ModernGL, puede ser que se parezca al formato de `struct.pack` pero es importante notar que no son iguales.

#### Ejemplos

Para crear un __VBO__ que contenga las cordenadas de puntos en _2D_, el codigo debe verse 
```python
# Cordenadas (x,y)
vertices = [
     0.0, 1.0,
    -1.0, 0.0,
     1.0, 0.0
]

# guardar en estructura y buffer
buffer = struct.pack('6f', vertices)
vbo = ctx.buffer(buffer)

# Unir VBO y VAO
vao = ctx.vertex_array(
    shader_program,
    [ (vbo, '2f', 'in_vert')],
    index_buffer_object
)
```
La linea despues del `shader_program` es la que indica el formato en que se debe leer el `vbo`. Indica que hay dos _floats_ que seran pasados a un atributo `in_vert` declarando en el _vertex shader_ como 
```
in vec2 in_vert;
```

### Porgram

Una de las principales caracteristicas de __ModernGL__ es que te permite definir tus propios shaders para renderizar cosas. Esto complica un poco las cosas pero te da la libertad para poder manipular la informacion como se desee.

> __Shader__ Programa que se ejecuta en la _GPU_

Antes de empezar a programar los _shaders_ es bueno notar que la esquina superior derecha de la pantalla corespone a las cordenadas $(1.0, 1.0)$ y, por lo tanto, la esquina inferior izquierda tiene las cordenadas $(-1.0, -1.0)$.

Para crear un `Program`, que es la manera en que __ModernGL__ maneja los _shaders_ se debe ligar a un contexto. Esto se hace con la linea
```python
program = ctx.program()
```

Este metodo recibe dos argumentos principales `shaders` y `varyings`, que corresponden a la lista de _shaders_ a usar y una lista de nombres variables. La lista de _shaders_ que puede recibir son
 - `vertex_shader`. El unico obligatorio
 - `fragment_shader`
 - `geometry_shader`
 - `tess_control_shader`
 - `tess_evaluation_shader`
 
A cada uno de estos argumentos se les debe pasar un _string_ que contenga los `shaders` a usar.

Un `vertex_shader` basico, que ponga cordendas 2D dadas en donde corresponda, se ve asi
```python
in vec2 in_vert;
in vec3 in_color;

out vec3 v_color;

void main(){
    v_volor = in_color;
    gl_Position = vec4(in_vert, 0.0, 1.0)
}
```
Y el `fragment_shader` que pase el color tal cual se ve
```python
in vec3 v_color;

out vec3 f_color;

void main(){
    f_color = v_color;
}
```

### Vertex Array

Para la creacion y manejo de los arreglos de vertices es recomendado usar la libreria `numpy` y sus metodos y tipos construidos.Unicamente hay que tener cuidado con la manera en que se pasan los datos al constructor del buffer. 

Para hacer esto es recomendable pasar un `numpy.ndarray` como el arreglo de datos, formateado con el metodo `astype('t')` (donde `t` debe ser cambiado por el tipo y tamanio deseado) y en formato de _bits_ de __C__, lo que se hace con el metodo `tobytes()`. Por lo que al final se puede construir el bufer con la sentencia

```python
ctx.buffer(vertices.astype('t').tobytes())
```
lo que nos regresara un __VBO__ para pasarle a
```python
ctx.simple_vertex_array(program, vbo, 'modo')
```
que nos dara el __VAO__. (__modo__ debe ser el conjunto de metodos para leer todo lo que este guardado en el buffer)


#### Ejemplo

Un ejemplo de esto podria ser que quisieramos guardar las cordenadas (x,y) y el respectivo color de cierta cantidad de puntos. Suponiendo que `x` sea un vector con _n_ elementos de las cordenadas _x_ de cada objeto, `y` el vector con las cordenadas _y_, `r` el valor del color en _red_, `g` en _verde_ y `b` en azul. Entonces esto se podria ver asi
```python
vertices = np.dstack([x, y, r, g, b])

vbo = ctx.buffer(vertices.astype('f4').tobytes())
vao = ctx.simple_vertex_array(prog, vbo, 'in_vert', 'in_color')
```

### Renderizar

Ahora toca, por fin, mostrar algo en pantalla.

In [25]:
import moderngl
import numpy as np

from PIL import Image

ctx = moderngl.create_standalone_context()

prog = ctx.program(
    vertex_shader='''
        #version 330

        in vec2 in_vert;
        in vec3 in_color;

        out vec3 v_color;

        void main() {
            v_color = in_color;
            gl_Position = vec4(in_vert, 0.0, 1.0);
        }
    ''',
    fragment_shader='''
        #version 330

        in vec3 v_color;

        out vec3 f_color;

        void main() {
            f_color = v_color;
        }
    ''',
)

x = np.linspace(-1.0, 1.0, 50)
y = np.random.rand(50) - 0.5
b = np.random.rand(50)
g = np.random.rand(50)
r = np.random.rand(50)

vertices = np.dstack([x, y, r, g, b])

vbo = ctx.buffer(vertices.astype('f4').tobytes())
vao = ctx.simple_vertex_array(prog, vbo, 'in_vert', 'in_color')

fbo = ctx.simple_framebuffer((512, 512))
fbo.use()
fbo.clear(0.0, 0.0, 0.0, 1.0)
vao.render(moderngl.LINE_STRIP)

Image.frombytes('RGB', fbo.size, fbo.read(), 'raw', 'RGB', 0, -1).show()

## Referencias